In [ ]:
import os
from langchain_community.document_loaders import GoogleDriveLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone


# API keys
os.environ["OPENAI_API_KEY"]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "google_credentials.json"

# Google Drive Loader
loader = GoogleDriveLoader(
    folder_id="15uTZMvgVgkosI0oflDwHADC7UxZYHGoS",
    recursive=True,
    file_types=["pdf", "document"]
)
documents = loader.load()

# Text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


import pinecone

os.environ['PINECONE_API_KEY'] = 'pcsk_wh6UM_NoELpceMNHsZoCwbztv6H3MDVesiLHEKpFPRmNgHs9QqyGjrkMPnXfwkY5mXgWg'

# Initialize Pinecone
pinecone_api_key = "pcsk_wh6UM_NoELpceMNHsZoCwbztv6H3MDVesiLHEKpFPRmNgHs9QqyGjrkMPnXfwkY5mXgWg"
pinecone_env = "us-east-1"
index_name = "llama-2-rag"

# Create Pinecone client
pc = pinecone.Pinecone(api_key="pcsk_wh6UM_NoELpceMNHsZoCwbztv6H3MDVesiLHEKpFPRmNgHs9QqyGjrkMPnXfwkY5mXgWg")

# Connect to my index
index = pc.Index(index_name)

# Vector store using OpenAI embeddings
embedding = OpenAIEmbeddings()
vectorstore = Pinecone.from_documents(docs, embedding=embedding, index_name=index_name)


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True
)

# Sample query
query = "What is Toronto?"
result = qa_chain({"query": query})

# Print the result
print("Answer:", result["result"])

Answer:  Toronto is a robot alien from Mars, a superhero, and a character from the comic book Action Comics #1.
